In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import mongoDB crud module
import aac_crud
from aac_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# username and password for mongoDB authentication
username = "Tom"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
# df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# encode Grazioso Salvare logo
image_filename = 'assets/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Insert the header and Grazioso Salvare logo
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard: Thomas Hotchkiss'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         style={'height':'15%', 'width':'15%'})),
    html.Hr(),
    
    # Dropdown for pre-set rescue type filter options
    html.Div([
        "Select Rescue Type:",
        dcc.Dropdown(['Water', 'Mountain/Wilderness', 'Human', 'Reset'], 'Reset', id='rescue_type',
                      style={'width': '400px'})
    ]),
        
    # Main data table
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         # Set up the features for your interactive data table to 
                         # make it user-friendly for your client
                         # Added sorting and filtering by column, and a limit of 10 results per page
                         sort_action='native',
                         filter_action='native',
                         sort_mode='multi',
                         page_action='native',
                         page_current= 0,
                         page_size= 10,
                         row_selectable='single',
                         selected_rows=[0]
                        ),
    html.Br(),
    html.Hr(),
    # Sets up the dashboard so that chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id', 'columns')],
              [Input('rescue_type', 'value')])
def update_dashboard(rescue_type):

    # find animals ideal for water rescues
    if (rescue_type == 'Water'):
        df = pd.DataFrame.from_records(db.read({
            'animal_type' : 'Dog', 
            'breed' : {'$in' :
                        ['Labrador Retriever Mix',
                         'Chesapeake Bay Retriever',
                         'Newfoundland']},
            'sex_upon_outcome' : 'Intact Female',
            'age_upon_outcome_in_weeks' : {'$gte' : 26, '$lte' : 156}
        }))
        
    # find animals ideal for mountain/wilderness rescues
    elif (rescue_type == 'Mountain/Wilderness'):
        df = pd.DataFrame.from_records(db.read({
            'animal_type' : 'Dog', 
            'breed' : {'$in' :
                        ['German Shepherd',
                         'Alaskan Malamute',
                         'Old English Sheepdog',
                         'Siberian Husky',
                         'Rottweiler']},
            'sex_upon_outcome' : 'Intact Male',
            'age_upon_outcome_in_weeks' : {'$gte' : 26, '$lte' : 156}
        }))
        
    # find animals ideal for human rescues
    elif (rescue_type == 'Human'):
        df = pd.DataFrame.from_records(db.read({
            'animal_type' : 'Dog', 
            'breed' : {'$in' :
                        ['Doberman Pinscher',
                         'German Shepherd',
                         'Golden Retriever',
                         'Bloodhound',
                         'Rottweiler']},
            'sex_upon_outcome' : 'Intact Male',
            'age_upon_outcome_in_weeks' : {'$gte' : 20, '$lte' : 300}
        }))
        
    # reset table to full dataset
    elif (rescue_type == 'Reset'):
        df = pd.DataFrame.from_records(db.read({}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)
    

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # checks if data table has valid data
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title='Proportion of Preferred Breeds')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


ModuleNotFoundError: No module named 'dash_leaflet'